In [48]:
from scipy.stats import norm
import numpy as np
import adaptfx as afx
import pandas as pd

In [49]:
def fit_normal_mean(sf_data):
    """
    fits the shape and scale parameters for a normal distribution

    Parameters
    ----------
    sf_data : array
        a nxk matrix with n the amount of patients and k the amount
        of sparing factors per patient

    Returns
    -------
    list
        mean and standard deviation hyperparameter
    """
    means = np.mean(sf_data, axis=1)
    mean, std = norm.fit(means)
    return [mean, std]

In [50]:
df_pre = pd.read_csv('data/sf_clean.csv', sep=',')
df = df_pre.loc[(df_pre['Patient'] < 31)]
adrenal = df.loc[(df['Target']=='Adrenal Glands') & (df['Limiting']==1)]
pancreas = df.loc[(df['Target']=='Pancreas') & (df['Limiting']==1)]
prostate = df.loc[(df['OAR']=='Rectum') & (df['Limiting']==1)]

In [51]:
def parameter_fit(data_sf):
    n_frac = int(data_sf['Fraction'].max() + 1)
    pre_sf_array = data_sf['sparing_factor'].to_numpy(dtype='float', copy=True)
    m = len(pre_sf_array)
    n_patients = int(m / n_frac)
    sf_array = np.reshape(pre_sf_array, (n_patients, n_frac))
    [mean_mean, std_mean] = fit_normal_mean(sf_array)
    [shape, scale] = afx.fit_gamma_prior(sf_array)
    return [mean_mean, std_mean, shape, scale]

In [52]:
parameter_fit(adrenal)

[0.6556666666666666,
 0.2720326777756264,
 6.4030496771870675,
 0.005188130993396964]

In [53]:
def get_variation_oar(data_sf, oar_target):
    print(data_sf.groupby(oar_target)['sparing_factor'].mean())
    patient_sf = data_sf.groupby(['Patient', oar_target])['sparing_factor'].std()
    patient_sf = patient_sf.reset_index(name='sparing_factor')
    print(patient_sf.groupby(oar_target)['sparing_factor'].mean())
    return None
get_variation_oar(pancreas, 'OAR')
get_variation_oar(adrenal, 'OAR')
get_variation_oar(prostate, 'Target')

OAR
Bowel       0.854444
Duodenum    0.820833
Stomach     0.884167
Name: sparing_factor, dtype: float64
OAR
Bowel       0.053322
Duodenum    0.030855
Stomach     0.050015
Name: sparing_factor, dtype: float64
OAR
Bowel       0.711111
Duodenum    0.586250
Stomach     0.692778
Name: sparing_factor, dtype: float64
OAR
Bowel       0.031708
Duodenum    0.043485
Stomach     0.031614
Name: sparing_factor, dtype: float64
Target
Boost       0.931667
Prostate    1.024333
Name: sparing_factor, dtype: float64
Target
Boost       0.015298
Prostate    0.014390
Name: sparing_factor, dtype: float64


In [ ]:
def sample_normal(mu, std, n, k):
    samples = np.random.normal(loc=mu, scale=std, size=(n,k))

    return samples

fit_samples = sample_normal(0.9, 0.04, 200, 6)

[shape_var, scale_var] = afx.fit_invgamma_prior(fit_samples)
[shape_std, scale_std] = afx.fit_invgamma_prior_std(fit_samples)

model_samples = sample_normal(0.9, 0.04, 1, 6)
afx.student_t(model_samples, shape_var, scale_var)
afx.student_t_std(model_samples, shape_std, scale_std)